# Expanding your machine learning toolkit: randomized search,  SVMs and regularized logistic regression

## Introduction

Previously, we wrote about some [common trade-offs](http://blog.cambridgecoding.com/2016/03/24/misleading-modelling-overfitting-cross-validation-and-the-bias-variance-trade-off/) in machine learning and the importance of [tuning models](http://blog.cambridgecoding.com/2016/04/03/scanning-hyperspace-how-to-tune-machine-learning-models/) to your specific dataset. We demonstrated tuning a **random forest** classifier using grid search, and how **cross-validation** can help avoid **overfitting** when tuning **hyperparameters**.

In this short follow-up post, we introduce a different strategy for traversing hyperparameter space - **randomized search**. We also demonstrate the process of tuning and training two other classification algorithms - a **support vector machine** and a **logistic regression classifier**.

![Algorithms we'll use in this tutorial.](hyperparam_algos_logistic.png)

We'll keep working with the [wine dataset](https://archive.ics.uci.edu/ml/datasets/Wine), which contains chemical characteristics of wines of varying quality. As before, our goal is to try to predict a wine's quality form these features.

Here are the things we'll cover in this blog post:

![Tutorial overview.](hyperparam_intro_logistic.png)

In the next blog post, you will learn how to take these three different tuned machine learning algorithms and combine them to build an aggregate **model ensemble**. Building ensembles often leads to improved model performance and generalizability.

## Loading and train/test splitting the dataset

You start off by collecting the dataset. We have covered the data loading, preprocessing,  and train/test splitting [previously](http://blog.cambridgecoding.com/2016/03/24/misleading-modelling-overfitting-cross-validation-and-the-bias-variance-trade-off/), so we won't repeat ourselves here. Also check out [this post](http://blog.cambridgecoding.com/2016/02/07/eda-and-interactive-figures-with-plotly/) on using plotly to create exploratory, interactive graphics of the wine dataset features. 

You can fetch and format the data as follows:

In [2]:
import wget
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

# Import the dataset
data_url = 'https://raw.githubusercontent.com/nslatysheva/data_science_blogging/master/datasets/wine/winequality-red.csv'
dataset = wget.download(data_url)
dataset = pd.read_csv(dataset, sep=";")

# Using a lambda function to bin quality scores
dataset['quality_is_high'] = dataset.quality.apply(lambda x: 1 if x >= 6 else 0)

# Convert the dataframe to a numpy array and split the
# data into an input matrix X and class label vector y
npArray = np.array(dataset)
X = npArray[:,:-2].astype(float)
y = npArray[:,-1]

# Split into training and test sets
XTrain, XTest, yTrain, yTest = train_test_split(X, y, random_state=1)

## Mixing things up: introducing randomized search

You have already built a random forest classifier, tuned using grid search, to predict wine quality ([here](http://localhost:8888/notebooks/polished_prediction/scanning_hyperspace.ipynb#)). Grid search is quite commonly used, and is essentially just a method that exhaustively tries out all manually prespecified HP values and reports the best option. 

Another way to search through hyperparameter space to find optima is by using **randomized search**. In randomized search, you sample HP values a certain number of times from some distribution which you prespecify in advance, which has the effect of easily narrowing down HP space to regions you think are sensible to explore. You also specify a `n_iter` parameter, which controls how many different parameter settings are tried out in total. There is [evidence](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf) that randomized search is more efficient than grid search, especially in high-dimensional feature spaces. 

If you were to sample from a uniform distribution n times and set `n_iter` to n, then randomized search would be practically (but not exactly - why?) equivalent to a grid search over equally spaced HP values.

You can feed distributions of HPs to the `RandomizedSearchCV` object in two (fairly similar) ways:
1. You can either define distributions over HPs, without immediately sampling from them, and pass these distributions to `RandomizedSearchCV`, which will proceed to sample `n_iter` number of times with replacement from the distributions to generate candidate HP combinations. 
2. You can sample from distributions immediately and pass a **list of possible HP values** to `RandomizedSearchCV`, and it will sample from these possible values `n_iter` number of times **without replacement**. 

Here is what the second approach looks like with random forests:

In [157]:
from scipy.stats import uniform
from scipy.stats import norm

from sklearn.grid_search import RandomizedSearchCV
from sklearn import metrics

# Designate distributions to sample hyperparameters from 
n_estimators = np.random.uniform(10, 45, 5).astype(int)
max_features = np.random.normal(6, 3, 5).astype(int)

# Check max_features>0 & max_features<=total number of features
max_features[max_features <= 0] = 1
max_features[max_features > X.shape[1]] = X.shape[1]

hyperparameters = {'n_estimators': list(n_estimators),
                   'max_features': list(max_features)}

print hyperparameters

{'n_estimators': [11, 20, 35, 12, 17], 'max_features': [9, 2, 8, 1, 6]}


We then run the random search:

In [132]:
from sklearn.ensemble import RandomForestClassifier

# Run randomized search
randomCV = RandomizedSearchCV(RandomForestClassifier(), param_distributions=hyperparameters, n_iter=10)
randomCV.fit(XTrain, yTrain)

# Identify optimal hyperparameter values
best_n_estim      = randomCV.best_params_['n_estimators']
best_max_features = randomCV.best_params_['max_features']  

print("The best performing n_estimators value is: {:5.1f}".format(best_n_estim))
print("The best performing max_features value is: {:5.1f}".format(best_max_features))

# Train classifier using optimal hyperparameter values
# We could have also gotten this model out from randomCV.best_estimator_
rf = RandomForestClassifier(n_estimators=best_n_estim,
                                max_features=best_max_features)

rf.fit(XTrain, yTrain)
rf_predictions = rf.predict(XTest)

print (metrics.classification_report(yTest, rf_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, rf_predictions),3))

The best performing n_estimators value is:  42.0
The best performing max_features value is:   2.0
             precision    recall  f1-score   support

        0.0       0.78      0.83      0.80       188
        1.0       0.84      0.79      0.82       212

avg / total       0.81      0.81      0.81       400

('Overall Accuracy:', 0.81)


Either grid search or randomized search is [probably fine](http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html) for tuning random forests.

Let's look at how to tune our two other predictors.

## Tuning a support vector machine model

Let's train our second algorithm, a **support vector machine** (SVM) classifier, to do the same prediction task. A great introduction to the theory behind SVMs can be found in Chapter 9 of the [Introduction to Statistical Learning book](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Sixth%20Printing.pdf) or [in this nice blog post](https://www.quantstart.com/articles/Support-Vector-Machines-A-Guide-for-Beginners). Briefly, SVMs search for **separating hyperplanes** in the feature space which best divide the different classes in your dataset. If you had 2 features, SVMs would search for the best dividing line; if you had 3 features, it would search for the best dividing 2d plane, etc. Crucially, SVMs can construct complex, **non-linear decision boundaries** between classes using a process called **kernelling**, which projects the data into a higher-dimensional space and facilitates the identification of a good boundary. This sounds a bit abstract, but if you've ever fit a straight line to power-transformed variables (e.g. maybe you used x but also x^2 and x^3 as predictors in a linear regression model), you're already familiar with the concept of creating higher dimensions to facilitate modelling in a lower dimensional setting.

SVMs can use different types of kernel functions, like linear, polynomial, Gaussian or radial kernels, to throw the data into a different space. Let's use the popular **radial basis kernel**. Of the available [hyperparameters](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), some key radial SVM hyperparameters to tune are:

+ `gamma` (a kernel parameter, controlling how 'far' we project the data into the new feature space) and
+ `C` (which controls the 'softness' of the classification boundary margin and hence the [bias-variance tradeoff](http://blog.cambridgecoding.com/2016/03/24/misleading-modelling-overfitting-cross-validation-and-the-bias-variance-trade-off/) of the SVM model

Examine the default HP settings and define your own: 

In [139]:
from sklearn import svm

default_SVC = svm.SVC()
print ("Default SVC parameters are:", default_SVC.get_params)

# Search for good hyperparameter values
# Specify values to grid search over
#g_range = 2. ** np.arange(-15, 5, 5)
#C_range = 2. ** np.arange(-5, 15, 5)
# Designate distributions to sample hyperparameters from 
g_range = np.random.uniform(0.0, 1, 10).astype(float)
C_range = np.random.normal(2, 1, 10).astype(float)
# Check that C>0 
print(C_range)
C_range[C_range < 0] = 0.001
print(C_range)

hyperparameters = {'gamma': list(g_range), 
                    'C': list(C_range)}

print (hyperparameters)

('Default SVC parameters are:', <bound method SVC.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)>)
[ 2.02569154  2.12224401  1.9941315   2.77637933  0.58742461  3.28107636
  0.88343967  1.71194508  3.13788987  3.64786504]
[ 2.02569154  2.12224401  1.9941315   2.77637933  0.58742461  3.28107636
  0.88343967  1.71194508  3.13788987  3.64786504]
{'C': [2.025691543323108, 2.1222440124808757, 1.9941314968377923, 2.7763793312528966, 0.58742461079524122, 3.2810763632600262, 0.88343966906169236, 1.7119450816468611, 3.1378898715478627, 3.647865044678789], 'gamma': [0.65075608328714085, 0.40566395285689916, 0.9095086787412392, 0.10999361209826486, 0.50522953223201705, 0.096035592909470613, 0.82786188243582737, 0.91852510593437908, 0.060956157006002876, 0.2619566287356947]}


In [142]:
from sklearn.svm import SVC

# Run randomized search
randomCV = RandomizedSearchCV(SVC(kernel='rbf'), param_distributions=hyperparameters, n_iter=20)
randomCV.fit(XTrain, yTrain)

# Identify optimal hyperparameter values
best_gamma      = randomCV.best_params_['gamma']
best_C          = randomCV.best_params_['C']  


print("The best performing gamma value is: {:5.2f}".format(best_gamma))
print("The best performing C value is: {:5.2f}".format(best_C))

The best performing gamma value is:  0.06
The best performing C value is:  3.65


In [143]:
# Train SVM and output predictions
rbfSVM = SVC(kernel='rbf', C=best_C, gamma=best_gamma)
rbfSVM.fit(XTrain, yTrain)
svm_predictions = rbfSVM.predict(XTest)

print (metrics.classification_report(yTest, svm_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, svm_predictions),3))

             precision    recall  f1-score   support

        0.0       0.67      0.77      0.71       188
        1.0       0.76      0.66      0.71       212

avg / total       0.72      0.71      0.71       400

('Overall Accuracy:', 0.71)


See how this compares to the default classifier:

In [144]:
rbfSVM = SVC(kernel='rbf')
rbfSVM.fit(XTrain, yTrain)
svm_predictions = rbfSVM.predict(XTest)

print (metrics.classification_report(yTest, svm_predictions))
print ("Overall Accuracy:", round(metrics.accuracy_score(yTest, svm_predictions),3))

             precision    recall  f1-score   support

        0.0       0.67      0.75      0.71       188
        1.0       0.75      0.67      0.71       212

avg / total       0.71      0.71      0.71       400

('Overall Accuracy:', 0.71)


## Tuning a logistic regression classifier

The final model you'll tune and apply to predict wine quality is a logistic regression classifier. This is a type of regression model which is used for predicting binary outcomes (like good wine/not good wine).  Logistic regression fits a sigmoidal (S-shaped) curve through the data, but can be viewed as just a transformed version of linear regression - a straight line predicting the [log odds](https://en.wikipedia.org/wiki/Logit) of data points being in one of the two classes is fit using maximum likelihood. A nice explanation of the theoretical basis for logistic regression can be found here.

One topic you will often encounter in machine learning is **regularization**, which is a class of techniques to reduce overfitting. The idea behind regularization is that you do not only want to maximize a model's fit to your data. This is liable to overfitting. Regularization techniques try to cut down on overfitting by penalizing models, for example if they use too many parameters, or if they assign coefficients or weights that are "too big". Regularization means that models have to learn from the data under a series of constraints, which often leads to robust representations of the data. You can read more about regularized regression [here]().

You can adjust just how much regularization you want by adjusting **regularization hyperparameters**, and since this is something you might want to do often, scikit-learn comes with some pre-built models that can very efficiently fit data for a range of regulatization hyperparameter values. This is the case for regularized linear regression models like [Lasso regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV) and [ridge regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html#sklearn.linear_model.RidgeCV), which use l1 and l2 penalties, respectively, to shrink the size of the regression coefficients. These scikit classes offer a shortcut to performing cross-validated selection of the regularization hyperparameter.

But you can also optimize how much regularization you want yourself, while at the same time tuning other hyperparameters (like the choice between l1 and l2 penalty), in the same manner as you've been doing:

In [152]:
# Tuning a regularized logistic regression model
from sklearn.linear_model import LogisticRegression

# Examine defaults
default_lr = LogisticRegression()
print ("Default logistic regression parameters are:", default_lr.get_params)
       
# Specify HP distributions
penalty = ["l1", "l2"]
C_range = np.random.normal(2, 1, 10).astype(float)

hyperparameters = {'penalty': penalty, 
                    'C': C_range}

print(hyperparameters)

('Default logistic regression parameters are:', <bound method LogisticRegression.get_params of LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)>)
{'penalty': ['l1', 'l2'], 'C': array([ 2.57784589,  0.36333581,  1.8194985 ,  3.10621018,  0.94523643,
        0.72607917,  2.60997877,  3.16702116,  2.08606353,  2.30047416])}


In [154]:
# Grid search using cross-validation
randomCV = RandomizedSearchCV(LogisticRegression(), param_distributions=hyperparameters, cv=10)  
randomCV.fit(XTrain, yTrain)

best_penalty = randomCV.best_params_['penalty']
best_C       = randomCV.best_params_['C']

print ("The best performing penalty is: {}".format(best_penalty))
print ("The best performing C value is: {:5.2f}".format(best_C))

The best performing penalty is: l1
The best performing C value is:  3.11


In [155]:
# Train model and output predictions
classifier_logistic = LogisticRegression(penalty=best_penalty, C=best_C)
classifier_logistic_fit = classifier_logistic.fit(XTrain, yTrain)
logistic_predictions = classifier_logistic_fit.predict(XTest)

print metrics.classification_report(yTest, logistic_predictions)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, logistic_predictions),3)

             precision    recall  f1-score   support

        0.0       0.72      0.75      0.73       188
        1.0       0.77      0.74      0.75       212

avg / total       0.75      0.74      0.75       400

Overall Accuracy: 0.745


Compare to default performance:

In [156]:
# Train model and output predictions
classifier_logistic = LogisticRegression()
classifier_logistic_fit = classifier_logistic.fit(XTrain, yTrain)
logistic_predictions = classifier_logistic_fit.predict(XTest)

print metrics.classification_report(yTest, logistic_predictions)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, logistic_predictions),3)

             precision    recall  f1-score   support

        0.0       0.73      0.74      0.73       188
        1.0       0.77      0.75      0.76       212

avg / total       0.75      0.75      0.75       400

Overall Accuracy: 0.748


## Conclusion

In this tutorial, you've X. Quick final comparison of grid vs. randomized search. The important thing is to do some exploration of HP space using either method.

Fancier techniques for hyperparameter optimization include methods based on [gradient descent](http://jmlr.org/proceedings/papers/v37/maclaurin15.pdf), grad student descent (not recommended), and [Bayesian approaches](http://arxiv.org/pdf/1206.2944.pdf) which update prior beliefs about likely values of hyperparameters based on the data (see [Spearmint](https://github.com/JasperSnoek/spearmint) and [hyperopt](http://hyperopt.github.io/hyperopt/)).

In the next post, you will take these different tuned models and build them up into an ensemble model with the aim of increasing predictive performance.